In [1]:
'''
This will create the data_W table for the case study.

@author: apschmidt2
'''
import csv, os
import pandas as pd
from collections import defaultdict
import numpy as np

# Which location are we making it for?
loc = 'Milwaukee' # 'Milwaukee' #


'''
The logic used is as follows:

Base Coverage:
A dropbox location covers an individual if at least one of the following is true
    a) The dropbox is within 15 minutes driving
    b) The dropbox is within 30 minutes transit
    c) The dropbox is within 15 minutes walking
    d) The dropbox is within 4 miles (road distance) to account for data quality
    issues and biking

Desired Coverage: 1/2 the values used within the base coverage

Each of these times are computed from the center of the block group or census
track considered.
'''
  
# These are the potential dropbox locations
db_locations = []
with open("C:/Users/apschmidt/Box/Research/Elections/Dropbox/Milwaukee Case Study Raw Data/potential_db_locations.csv",'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    count = 0
    for row in reader:
        if count == 0:
            count += 1
        else:
            db_locations.append(row[0])

# -----------------------------------------------
# We first read in the locations and store the travel times
# -----------------------------------------------
BG_traveltimes = defaultdict(lambda: defaultdict(dict)) # Will store the travel times for each block group to each dropbox location
Tract_traveltimes = defaultdict(lambda: defaultdict(dict)) # Will store the travel times for each census tract to each dropbox location
BG_traveldistances =  defaultdict(lambda: defaultdict(dict)) # Will store the travel distance for each block group to each dropbox location
center_directory = 'C:/Users/apschmidt/Box/Research/Elections/Dropbox/Milwaukee Case Study Raw Data/Census Data' 
duration_directory = 'C:/Users/apschmidt/Box/Research/Elections/Dropbox/Milwaukee Case Study Raw Data/Distance Tables'


# First, we deal with the block groups. This reads in the block groups that are
# within the city limits and within milwaukee county
with open(os.path.join(center_directory, 'milwaukee_block_group_list.txt'), 'r') as f:
    output = f.readlines()
    list_of_blockgroups = [line.rstrip() for line in output]
    
# Now, reading in the file to get the FIPS code
df = pd.read_csv(os.path.join(center_directory, 'blockgroupCenters.csv'))
# key = OBJECTID, value = FIPS code
FIPS_codes = pd.Series(df.FIPS_BLKGR.values,index=df.OBJECTID).to_dict()


# Now, we read in the distance between each drop box location
for tm in ['driving','walking','transit']:
    with open(os.path.join(duration_directory,f'voterblockgroup_{tm}_duration_matrix.csv'), 'r', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        count = 0
        for row in spamreader:
            if count == 0:
                count += 1
                dbs = row # Stores the dropboxes (plus the 0th index)
            else:
                vl = int(row[0])# Voter location
                for i in range(1,len(row)):
                    if float(row[i]) < 0:
                        row[i] = 9999
                    BG_traveltimes[FIPS_codes[vl]][dbs[i]][tm] = float(row[i])
    
# Reading in the road distance
with open(os.path.join(duration_directory,f'voterblockgroup_driving_distance_matrix.csv'), 'r', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        count = 0
        for row in spamreader:
            if count == 0:
                count += 1
                dbs = row # Stores the dropboxes (plus the 0th index)
            else:
                vl = int(row[0])# Voter location
                for i in range(1,len(row)):
                    if float(row[i]) < 0:
                        row[i] = 9999
                    BG_traveldistances[FIPS_codes[vl]][dbs[i]] = float(row[i])

# We now repeat for the census tracts.

# Now, reading in the file to get the FIPS code
df = pd.read_csv(os.path.join(center_directory, 'tractCenters.csv'))
# key = OBJECTID, value = FIPS code
FIPS_codes = pd.Series(df.FIPS_TRACT.values,index=df.OBJECTID).to_dict()


# Now, we read in the distance between each drop box location
for tm in ['driving','walking','transit']:
    with open(os.path.join(duration_directory,f'votertract_{tm}_duration_matrix.csv'), 'r', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        count = 0
        for row in spamreader:
            if count == 0:
                count += 1
                dbs = row # Stores the dropboxes (plus the 0th index)
            else:
                vl = int(row[0])# Voter location
                for i in range(1,len(row)):
                    if float(row[i]) < 0:
                        row[i] = 9999
                    Tract_traveltimes[FIPS_codes[vl]][dbs[i]][tm] = float(row[i])



#########
# Coverage
########
'''
We define coverage sets for each block group locations. Each location should
be covered equally.
'''
                    
def find_basecoverage(FIPS_BLOCK_GROUP):
    '''
    This takes in a row of our data frame and determines which of the 
    drop box locations will provide the base coverage
    '''
    covering_locations = []
    
    
    # Can only look at transit and walking times
    for db in db_locations:
        count = 0
        # Residence
        if BG_traveltimes[FIPS_BLOCK_GROUP][db]['walking'] <= 15:
            count += 1
        # Residence
        if BG_traveltimes[FIPS_BLOCK_GROUP][db]['transit'] <= 30:
            count += 1
        # We can check the drive time
        if BG_traveltimes[FIPS_BLOCK_GROUP][db]['driving'] <= 15:
            count += 1
        # Work location
        if BG_traveldistances[FIPS_BLOCK_GROUP][db] <= 4:
            count += 1
        if count >= 2:
            covering_locations.append(db)
    if len(covering_locations) < 2:
        print(FIPS_BLOCK_GROUP)
        print(BG_traveltimes[FIPS_BLOCK_GROUP])
        print(BG_traveldistances[FIPS_BLOCK_GROUP])
        print(covering_locations)
        raise Exception('Issue')
    return '-'.join(covering_locations)
        
def find_coverage(FIPS_BLOCK_GROUP):
    '''
    This takes in a row of our data frame and determines which of the 
    drop box locations will provide the desired coverage
    '''
    covering_locations = []
    
    # Can only look at transit and walking times
    for db in db_locations:
        # Residence
        if BG_traveltimes[FIPS_BLOCK_GROUP][db]['walking'] <= 15/2:
            covering_locations.append(db)
        # Residence
        elif BG_traveltimes[FIPS_BLOCK_GROUP][db]['transit'] <= 30/2:
            covering_locations.append(db)
        # We can check the drive time
        elif BG_traveltimes[FIPS_BLOCK_GROUP][db]['driving'] <= 15/2:
            covering_locations.append(db)
        # Work location
        elif BG_traveldistances[FIPS_BLOCK_GROUP][db] <= 4/2:
            covering_locations.append(db)
            
    return '-'.join(covering_locations)

df = pd.read_csv(os.path.join(center_directory,'consolidated_blockgroup_data.csv'))
# df = df[df['18+ Population 2020'] > 0]

map_bg_pop = pd.Series(df['18+ Population 2020'].values,index=df.FIPS_BLOCK_GROUP).to_dict()

# Now writing coverage to file
with open(f'../MCTP/Data/{loc}/data_W.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['ID','w','p','Ni','Nprime'])
    for bg in map_bg_pop:
#         if map_bg_pop[bg] > 0:
        spamwriter.writerow([bg,1,map_bg_pop[bg],find_coverage(bg),find_basecoverage(bg)])
    


##########
# Now, we construct an intermediary data set and save it. This will be used for
# The participation
##########


# Reading in the population, voting rate and vehicle ownership for locations
df = pd.read_csv(os.path.join(center_directory,'consolidated_blockgroup_data.csv'))
# df = df[df['18+ Population 2020'] > 0]
df = df[['FIPS_BLOCK_GROUP','18+ Population 2020', 'Nov 2020 Turnout Rate (Est)','No vehicle available']]
# df['Nov 2020 Turnout Rate (Est)'] = df['Nov 2020 Turnout Rate (Est)'].clip(lower = 50)

df = df.rename(columns = {'No vehicle available':'Vehicle Percent'})
df_someno = df[df['Vehicle Percent'] > 0].reset_index()
df['Vehicle Percent'] = 1-df['Vehicle Percent']
df['Vehicle Status'] = 'Available'
df_someno['Vehicle Status'] = 'NoneAvailable'
df = df.append(df_someno, ignore_index = True).drop(columns = ['index']).sort_values('FIPS_BLOCK_GROUP').reset_index(drop = True)

df_copy = df.copy() # We need a copy for non-workers/work outside MKE
df_copy['Work Location'] = 'No work/Outside MKE'
df_copy['FIPS_TRACT'] = 'NA'
df['Work Location'] = 'In MKE'


# Now reading in the worker location data
# First, adding column to our dataframe to store the percent of people working
# need to do this so percentages work out correctly later
df_workrate = pd.read_csv(os.path.join(center_directory,f'Residence to Workplace/flow_table.csv'))
df_workrate = df_workrate.rename(columns = {'v FIPS_BLOCK_GROUP - FIPS_TRACT ->':'FIPS_BLOCK_GROUP'})


df_workrate_mke = df_workrate.copy()
df_workrate_mke['Non MKE Worker'] = df_workrate['Total']- df_workrate[[str(i) for i in Tract_traveltimes if str(i) in df_workrate.columns]].sum(axis = 1)
df_workrate_mke = df_workrate_mke[['FIPS_BLOCK_GROUP','Non MKE Worker']]
df_copy = df_copy.merge(df_workrate_mke, on = 'FIPS_BLOCK_GROUP', how = 'left')
df_copy['Work Percent'] = df_copy['Non MKE Worker']/df['18+ Population 2020']
df_copy['Work Percent'] = df_copy['Work Percent'].clip(upper = 1)
df_copy['Percent Work Location'] = 1
df_copy = df_copy.drop(columns  = ['Non MKE Worker'])

df_workrate = df_workrate[['FIPS_BLOCK_GROUP','Total']]
df = df.merge(df_workrate, on = 'FIPS_BLOCK_GROUP', how = 'left') 
df['Work Percent'] = df['Total']/df['18+ Population 2020']    
df['Work Percent'] = df['Work Percent'].clip(upper = 1)    


# Now reading in the data from the flow table
df_flow = pd.read_csv(os.path.join(center_directory,f'Residence to Workplace/flow_table.csv'))
df_flow = df_flow.drop(columns = ['Total','Out of State'])
df_flow = df_flow.rename(columns = {'v FIPS_BLOCK_GROUP - FIPS_TRACT ->':'FIPS_BLOCK_GROUP'})
df_flow = df_flow.melt(id_vars=["FIPS_BLOCK_GROUP"], 
        var_name="FIPS_TRACT", 
        value_name="Number Workers")
df_flow["FIPS_TRACT"] = df_flow["FIPS_TRACT"].astype(np.int64)

# Only consider work location within milwaukee for simplicity
df_flow = df_flow[df_flow['FIPS_TRACT'].isin(list(Tract_traveltimes.keys()))]
df_flow = df_flow[df_flow['Number Workers']>0]

df = df.merge(df_flow, on = 'FIPS_BLOCK_GROUP', how = 'left') 
df['Percent Work Location'] = df["Number Workers"]/df['Total']
df['Percent Work Location'] = df['Percent Work Location'].clip(upper = 1)
df = df.drop(columns = ['Total'])
df = df.drop(columns = ['Number Workers'])

### Now cleaning everything up

df = df.append(df_copy,ignore_index = True)
df['Percent of BG Population'] = df['Vehicle Percent']*df['Work Percent']*df['Percent Work Location']
df['Estimated Population'] = df['Percent of BG Population']*df['18+ Population 2020']

df = df[['FIPS_BLOCK_GROUP', 'FIPS_TRACT', 'Vehicle Status', 
       'Nov 2020 Turnout Rate (Est)', 'Estimated Population']]

# df = df[df['Estimated Population'] > 0]


################################
# Now for each BG residence - Tract POW - vehicle availability pairing, 
# determining which drop boxes cover the location
################################

# First, merge the distance tables for each pairing

def find_dropboxduration(row,dropbox):
    '''
    This will create a string to represent the 
    driving/transit/walking/work-walking duration to the dropbox location
    '''
    
    s = ""
    if row['Vehicle Status'] != 'NoneAvailable':
        s += str(BG_traveltimes[row['FIPS_BLOCK_GROUP']][db]['driving'])
        s+='-'
    else:
        s+='9999-'
    s += str(BG_traveltimes[row['FIPS_BLOCK_GROUP']][db]['transit'])+'-'
    s += str(BG_traveltimes[row['FIPS_BLOCK_GROUP']][db]['walking'])+'-'
    s += str(BG_traveldistances[row['FIPS_BLOCK_GROUP']][db])+'-'
    
    if row['FIPS_TRACT'] != 'NA':
        s += str(Tract_traveltimes[row['FIPS_TRACT']][db]['walking'])
    else:
        s += '9999'
    
    
    return s
        
df['coverage_base'] = df.apply(lambda x: find_basecoverage(x['FIPS_BLOCK_GROUP']), axis=1)
df['coverage'] = df.apply(lambda x: find_coverage(x['FIPS_BLOCK_GROUP']), axis=1)
for db in db_locations:
    df[db] = df.apply(lambda x: find_dropboxduration(x,db), axis=1)

print(df)

df.to_csv('C:/Users/apschmidt/Box/Research/Elections/Dropbox/Milwaukee Case Study Raw Data/Clean Data Set/20211118_data.csv',index = False)
print("Wrote intermediate file")





        FIPS_BLOCK_GROUP   FIPS_TRACT Vehicle Status  \
0           550790001011  55079186800      Available   
1           550790001011  55079187400      Available   
2           550790001011  55079020000      Available   
3           550790001011  55079160300      Available   
4           550790001011  55079185300      Available   
...                  ...          ...            ...   
130547      550791870003           NA      Available   
130548      550791870003           NA  NoneAvailable   
130549      550791874001           NA      Available   
130550      550791874001           NA  NoneAvailable   
130551      550799800001           NA      Available   

        Nov 2020 Turnout Rate (Est)  Estimated Population  \
0                         78.550000              1.381616   
1                         78.550000              5.526462   
2                         78.550000              2.763231   
3                         78.550000              0.690808   
4                     

Wrote intermediate file
Wrote participation file


In [130]:

# Now, we are figuring out the participation parameters
def participation_params(row):
    '''
    This makes a new dataframe row that will have the necessary parameters
    for the choice model
    '''
    row_new = row.copy()
    turnout = row['Nov 2020 Turnout Rate (Est)']
    row_new['Location_ID'] = str(row['FIPS_BLOCK_GROUP'])+"-"+str(row['Vehicle Status'])+"-"+str(row['FIPS_TRACT'])
    
    num = 1
    for i,v in row_new.items():
        if i not in ['Location_ID','FIPS_BLOCK_GROUP','FIPS_TRACT','Vehicle Status','Estimated Population','coverage_base','coverage','Nov 2020 Turnout Rate (Est)']:
            drive, transit, walking, road_distance, work = v.split('-')
            row_new[i] = num/float(transit)**4+num/float(walking)**4+num/(float(road_distance)/15*60)**4
            if row['Vehicle Status'] == 'Available':
                row_new[i] = row_new[i] + num/float(drive)**4
            
            if row['FIPS_TRACT'] != 'NA':
                row_new[i] = row_new[i]+num/float(work)**4
            row_new[i] = 5*row_new[i]**(1/4)
            row_new[i] = row_new[i]*(100/float(turnout))**2
    row_new['None'] = max(100-turnout,0.0001)
    row_new['Other'] = min(turnout,100)
    return row_new

df_new = df.apply(lambda x: participation_params(x), axis=1)
# row_new = row_new.drop(labels = ['FIPS_BLOCK_GROUP','FIPS_TRACT','Vehicle Status','Estimated Population','coverage_base','coverage','Nov 2020 Turnout Rate (Est)'])
    
# df_new.to_csv('C:/Users/apschmidt/Box/Research/Elections/Dropbox/Code/MCTP/Data/Milwaukee/data_participation.csv',index = False)
# print("Wrote participation file")

In [131]:
# list(df_new.columns)

db_locations = ['Court1',
 'Court2',
 'Court3',
 'Court4',
 'Elect1',
 'Elect2',
 'Fire1',
 'Fire2',
 'Fire4',
 'Fire6',
 'Fire7',
 'Fire8',
 'Fire9',
 'Fire10',
 'Fire11',
 'Fire12',
 'Fire13',
 'Fire14',
 'Fire16',
 'Fire18',
 'Fire21',
 'Fire22',
 'Fire23',
 'Fire24',
 'Fire26',
 'Fire27',
 'Fire29',
 'Fire30',
 'Fire32',
 'Fire33',
 'Fire34',
 'Fire35',
 'Fire36',
 'Fire37',
 'Fire38',
 'Fire39',
 'Library1',
 'Library2',
 'Library3',
 'Library4',
 'Library5',
 'Library6',
 'Library7',
 'Library8',
 'Library9',
 'Library10',
 'Library11',
 'Library12',
 'Library13',
 'Library14',
 'Police1',
 'Police2',
 'Police3',
 'Police4',
 'Police5',
 'Police6',
 'Police7',
 'CVS1',
 'CVS2',
 'CVS3',
 'CVS4',
 'CVS5',
 'CVS6',
 'CVS7',
 'Walgreens1',
 'Walgreens2',
 'Walgreens3',
 'Walgreens4',
 'Walgreens5',
 'Walgreens6',
 'Walgreens7',
 'Walgreens8',
 'Walgreens9',
 'Walgreens10',
 'Walgreens11',
 'Walgreens12',
 'Walgreens13',
 'Walgreens14',
 'Walgreens15',
 'Walgreens16',
 'Walgreens17',
 'Walgreens18',
 'Walgreens19',
 'Walgreens20',
 'Walgreens21',
 'Walgreens22',
 'Walgreens23',
 'Walgreens24',
 'Walgreens25',
 'Walgreens26',
 'Walgreens27',
 'Walgreens28',
 'Walgreens29']

In [132]:
df_new = df_new.sort_values(by = ['FIPS_BLOCK_GROUP']+db_locations,ascending= True)

locations = df_new['Location_ID'].unique()

# Note that each BG location must have at least one, so we can compare within blockgroups
block_groups_dict = dict(zip(df_new['Location_ID'], df_new['FIPS_BLOCK_GROUP']))
vNone = dict(zip(df_new['Location_ID'], df_new['None']))
vDB = {}
for db in db_locations:
    vDB[db] = dict(zip(df_new['Location_ID'], df_new[db]))

In [133]:
keep_rows = []
remove_rows = {}
for i in locations:
    if i not in remove_rows:
        keep_rows.append(i)
        # Checking to see if i dominates any j
        for j in np.unique(df_new[df_new['FIPS_BLOCK_GROUP'] == block_groups_dict[i]]['Location_ID']):
            # Note that each block group location will have the same non-voter rate, so we don't need to check
            dominated = True
            for db in vDB:
                if vDB[db][i] > vDB[db][j]:
                    dominated = False
                    break
            if dominated:
                remove_rows[j] = True


In [134]:
len(keep_rows)

583

In [135]:
keep_rows

['550790001011-NoneAvailable-NA',
 '550790001012-NoneAvailable-NA',
 '550790001013-NoneAvailable-NA',
 '550790001014-NoneAvailable-NA',
 '550790001021-NoneAvailable-NA',
 '550790001022-NoneAvailable-NA',
 '550790001023-NoneAvailable-NA',
 '550790002011-NoneAvailable-NA',
 '550790002012-NoneAvailable-NA',
 '550790002013-NoneAvailable-NA',
 '550790002021-NoneAvailable-NA',
 '550790002022-NoneAvailable-NA',
 '550790002023-NoneAvailable-NA',
 '550790003011-NoneAvailable-NA',
 '550790003021-NoneAvailable-NA',
 '550790003031-Available-NA',
 '550790003032-NoneAvailable-NA',
 '550790003041-Available-NA',
 '550790003042-NoneAvailable-NA',
 '550790004001-NoneAvailable-NA',
 '550790004002-NoneAvailable-NA',
 '550790005011-NoneAvailable-NA',
 '550790005012-NoneAvailable-NA',
 '550790005013-NoneAvailable-NA',
 '550790005021-Available-NA',
 '550790005022-NoneAvailable-NA',
 '550790005023-NoneAvailable-NA',
 '550790006001-NoneAvailable-NA',
 '550790006002-NoneAvailable-NA',
 '550790006003-NoneAvailab

In [136]:
df_new2 = df_new.drop(columns = ['FIPS_BLOCK_GROUP','FIPS_TRACT','Vehicle Status','Estimated Population','coverage_base','coverage','Nov 2020 Turnout Rate (Est)']) 
df_new2 = df_new2[df_new2['Location_ID'].isin(keep_rows)]
df_new2= df_new2.set_index('Location_ID')
df_new2.to_csv('C:/Users/apschmidt/Box/Research/Elections/Dropbox/Code/MCTP/Data/Milwaukee/data_participation.csv')
print("Wrote participation file")

PermissionError: [Errno 13] Permission denied: 'C:/Users/apschmidt/Box/Research/Elections/Dropbox/Code/MCTP/Data/Milwaukee/data_participation.csv'

In [96]:
df_new2

Court1    Court2    Court3    Court4    Elect1    Elect2     Fire1  \
129423  0.063067  0.063855  0.064077  0.055498  0.055996  0.050323  0.055951   
129425  0.072822  0.072350  0.073087  0.066665  0.066536  0.059511  0.066275   
129427  0.069846  0.070815  0.071491  0.062399  0.063240  0.058427  0.063423   
129429  0.074361  0.073734  0.073950  0.066447  0.066891  0.059563  0.066604   
129431  0.073158  0.073191  0.074477  0.066077  0.065946  0.059479  0.065712   
...          ...       ...       ...       ...       ...       ...       ...   
130543  0.263873  0.311457  0.303966  0.424565  0.370775  0.194312  0.361092   
130546  0.339026  0.378793  0.367591  0.492076  0.529975  0.199420  0.535955   
130548  0.274709  0.342303  0.329166  0.472869  0.481685  0.204316  0.403709   
130550  0.516399  0.516546  0.496026  1.010603  0.949443  0.626128  0.958476   
130551  0.415148  0.424330  0.421983  0.689876  0.564198  0.315681  0.577339   

           Fire2     Fire4     Fire6  ...  Walgreens23  Walgreens24  \
129423  0.062019  0.133022  0.058335  ...     0.121777     0.060113   
129425  0.072086  0.154457  0.067533  ...     0.143392     0.065255   
129427  0.069614  0.136190  0.065906  ...     0.143251     0.061946   
129429  0.073013  0.144146  0.067610  ...     0.152392     0.063935   
129431  0.071922  0.158056  0.067157  ...     0.169648     0.066152   
...          ...       ...       ...  ...          ...          ...   
130543  0.286892  0.075221  0.856691  ...     0.115034     0.083322   
130546  0.380758  0.083904  1.679154  ...     0.107318     0.084967   
130548  0.307021  0.081457  1.206573  ...     0.096413     0.086570   
130550  0.606050  0.088234  0.431203  ...     0.106098     0.100279   
130551  0.445817  0.152563  0.903245  ...     0.168671     0.180777   

        Walgreens25  Walgreens26  Walgreens27  Walgreens28  Walgreens29  \
129423     0.024992     0.138936     0.107439     0.186714     0.217082   
129425     0.046046     0.162339     0.191657     0.271282     0.251490   
129427     0.045705     0.140512     0.174316     0.239468     0.189508   
129429     0.046877     0.147681     0.193392     0.264172     0.199356   
129431     0.046612     0.165825     0.197300     0.317020     0.244679   
...             ...          ...          ...          ...          ...   
130543     0.084294     0.080345     0.105109     0.088859     0.069437   
130546     0.083695     0.082842     0.099393     0.087616     0.066103   
130548     0.086042     0.080342     0.103081     0.087539     0.067413   
130550     0.108083     0.086821     0.093660     0.082486     0.069703   
130551     0.187871     0.145413     0.175677     0.144579     0.133939   

                          Location_ID       None      Other  
129423  550790001011-NoneAvailable-NA  21.450000  78.550000  
129425  550790001012-NoneAvailable-NA  21.450000  78.550000  
129427  550790001013-NoneAvailable-NA  22.760000  77.240000  
129429  550790001014-NoneAvailable-NA  22.760000  77.240000  
129431  550790001021-NoneAvailable-NA  20.230000  79.770000  
...                               ...        ...        ...  
130543  550791870001-NoneAvailable-NA  18.829195  81.170805  
130546  550791870002-NoneAvailable-NA  21.120000  78.880000  
130548  550791870003-NoneAvailable-NA  21.093542  78.906458  
130550  550791874001-NoneAvailable-NA  17.191983  82.808017  
130551      550799800001-Available-NA  19.878966  80.121034  

[583 rows x 96 columns]

In [100]:
keep_rows = []
remove_rows = {}
for i in locations:
    if i not in remove_rows:
        keep_rows.append(i)
        # Checking to see if i dominates any j
        for j in locations:
            if j not in remove_rows:
                # Note that each block group location will have the same non-voter rate, so we don't need to check
                dominated = True
                if vNone[j] > vNone[i]:
                    dominated = False
                for db in vDB:
                    if vDB[db][i] > vDB[db][j]:
                        dominated = False
                        break
                if dominated:
                    remove_rows[j] = True

In [101]:
len(keep_rows)

544